In [1]:
pip install --upgrade pyod



[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install combo


[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip uninstall torch

In [4]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import os.path as osp
import time
import random
import numpy as np
import torch
from parse2 import parse_arguments
import sklearn.model_selection
import torchdiffeq
from sklearn import metrics 
import pathlib

In [6]:
torch.cuda.is_available()

True

In [7]:
pip install torch_geometric


[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torchdiffeq


[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install torchsde



[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import ruptures as rpt
from collections import Counter

In [11]:
# columns = ['sensors.battery.temp',
#        'sensors.battery.percent', 'sensors.battery.solarmv',
#        'sensors.battery.current', 'sensors.battery.voltage',
#        'sensors.battery.solarma', 'sensors.battery.charging',
#         'solar_power', 'battery_power','power_difference']

In [12]:
# # Reading back and setting the MultiIndex
# labelled_data = pd.read_feather('../MTDataBatch1\Labelled Data\Labelled Batch 1 - Updated.feather')
# labelled_data.set_index(['DeviceId', 'time'], inplace=True)
# # labelled_data

In [13]:
# labelled_data

In [14]:
# labelled_data = labelled_data.sort_index(level=['DeviceId', 'time'])

In [15]:
# numberOfDevices = len(labelled_data.groupby(level=0))
# numberOfDevices

In [16]:
# column_list = list(labelled_data.columns)
# for column in column_list:
#     print(column)
#     print(labelled_data[column].isnull().values.any())
#     print("-------------------")

In [17]:
# len(labelled_data.columns)

In [18]:
# train_size = int(len(labelled_data) * 0.80)
# test_size = len(labelled_data) - train_size

In [19]:
# # Training data : take all rows, and only device feature columns (From "sensors.battery.extpower" to "power_difference")
# train_data = labelled_data.iloc[0:train_size,6:29]
# test_data = labelled_data.iloc[train_size:len(labelled_data),6:29]
# # Drop columns that have string values, or even NaN values 
# # Manual Inspection proved them as non important for malfunction detectio.
# train_data = train_data.drop('location.source', axis=1)
# train_data = train_data.drop('location.numSats', axis=1)
# train_data = train_data.drop('sensors.accelerometer.y', axis=1)
# train_data = train_data.drop('sensors.accelerometer.x', axis=1)
# train_data = train_data.drop('sensors.accelerometer.z', axis=1)
# test_data = test_data.drop('location.source', axis=1)
# test_data = test_data.drop('location.numSats', axis=1)
# test_data = test_data.drop('sensors.accelerometer.y', axis=1)
# test_data = test_data.drop('sensors.accelerometer.x', axis=1)
# test_data = test_data.drop('sensors.accelerometer.z', axis=1)
# # Labels is the array of values in "isDeviceWorking" -  convert to array of inetgers (0 or 1 values)
# labels = labelled_data.iloc[:,29].values


In [20]:
# len(train_data.columns)

In [21]:
# # Timestamps of devices that are working
# train_data_label.loc[train_data_label['isDeviceWorking'] == 1]

In [22]:
# deviceWorkingLabels = labelled_data[['isDeviceWorking']].copy()
# deviceWorkingLabels['isDeviceWorking'] = deviceWorkingLabels['isDeviceWorking'].astype(int)
# deviceWorkingLabels

In [23]:
# train_data_label = deviceWorkingLabels.iloc[0:train_size,:]
# test_data_label = deviceWorkingLabels.iloc[train_size:len(deviceWorkingLabels),:]

In [24]:
# train_data_label

In [25]:
# # Scale column values (without normalizing the indexes)
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(-1,1))

# # Group by level 0, which is the first index corresponding to 'DeviceId'
# # After applying the scaling function (which output a Numpy array), convert it to a dataframe

# normalized_train_data = train_data.groupby(level=0).apply(lambda x : pd.DataFrame(scaler.fit_transform(x), columns=x.columns, index=x.index).round(5))
    
# normalized_test_data = test_data.groupby(level=0).apply(lambda x : pd.DataFrame(scaler.fit_transform(x), columns=x.columns, index=x.index).round(5))

# normalized_test_data

In [26]:
# testdeviceIDs = set(normalized_test_data.index.get_level_values(0).tolist())
# print(testdeviceIDs)

In [27]:
# normalized_train_data[normalized_train_data.index.get_level_values('DeviceId') == '015262001277846']

In [28]:
def _pad(channel, maxlen):
    
    channel = torch.tensor(channel) # Series
    out = torch.full((maxlen,), channel[-1]) # tensor
    out[:channel.size(0)] = channel 
    return out 

In [29]:
def _process_data(look_window,forecast_window,stride_window,missing_rate,loc,learning_method):
    PATH = os.path.dirname(os.path.abspath(__file__))
    from sklearn import preprocessing
    torch.__version__
    
    data_path = PATH + '/data'
    
    X_times = normalized_train_data
    y_train_label = train_data_label
    X_times_test = normalized_test_data
    y_label = test_data_label
    input_size = X_times.shape[1]
    X_times = time_dataset.normalize(X_times)
    X_times_test = time_dataset.normalize(X_times_test)
    y_train_label = y_train_label[:X_times.shape[0]]
    total_length = len(X_times)
    
    timelen = X_times.shape[0]
    timelen_test = X_times_test.shape[0]
    
    full_seq_data_train = torch.Tensor()
    forecast_seq_train = torch.Tensor()
    full_y_seq_data_train = torch.Tensor()
    full_forecast_data_train = torch.Tensor()
    full_seq_data_test = torch.Tensor()
    full_y_seq_data = torch.Tensor()
    full_forecast_data_test = torch.Tensor()
    forecast_seq_test = torch.Tensor()
    
    # TRAIN 
    for _len in range(int((timelen-look_window+stride_window-forecast_window)/stride_window)):
        full_seq_temp = torch.Tensor([X_times[(_len*stride_window):(_len*stride_window)+look_window]])
        forecast_seq_temp = torch.Tensor([X_times[(_len*stride_window)+look_window:(_len*stride_window)+look_window+forecast_window]])

        full_y_seq_temp = torch.Tensor([y_train_label[(_len*stride_window):(_len*stride_window)+look_window]])
        forecast_y_seq_temp = torch.Tensor([y_train_label[(_len*stride_window)+look_window:(_len*stride_window)+look_window+forecast_window]])

        full_seq_data_train=torch.cat([full_seq_data_train,full_seq_temp])
        forecast_seq_train = torch.cat([forecast_seq_train,forecast_seq_temp])
        full_y_seq_data_train = torch.cat([full_y_seq_data_train,full_y_seq_temp])
        full_forecast_data_train = torch.cat([full_forecast_data_train,forecast_y_seq_temp])
        
    if missing_rate ==0:

        DATA_PATH_SAVE =PATH + '/processed_data/look_'+str(look_window)+'_stride_'+str(stride_window)+'_forecast_'+str(forecast_window)
    else:
        DATA_PATH_SAVE =PATH + '/processed_data/look_'+str(look_window)+'_stride_'+str(stride_window)+'_forecast_'+str(forecast_window)+'_Missing_'+str(missing_rate)

    # TEST
    for _len in range(int((timelen_test-look_window+stride_window-forecast_window)/stride_window)):
        # import pdb ; pdb.set_trace()
        full_seq_temp = torch.Tensor([X_times_test[(_len*stride_window):(_len*stride_window)+look_window]])
        forecast_seq_temp = torch.Tensor([X_times_test[(_len*stride_window)+look_window:(_len*stride_window)+look_window+forecast_window]])
        
        full_y_seq_temp = torch.Tensor([y_label[(_len*stride_window):(_len*stride_window)+look_window]])
        forecast_y_seq_temp = torch.Tensor([y_label[(_len*stride_window)+look_window:(_len*stride_window)+look_window+forecast_window]])
        
        full_seq_data_test=torch.cat([full_seq_data_test,full_seq_temp])
        forecast_seq_test = torch.cat([forecast_seq_test,forecast_seq_temp])
        full_y_seq_data = torch.cat([full_y_seq_data,full_y_seq_temp])
        full_forecast_data_test = torch.cat([full_forecast_data_test,forecast_y_seq_temp])
    # import pdb ; pdb.set_trace()
    if missing_rate != 0:
        generator = torch.Generator().manual_seed(56789)
        # import pdb ; pdb.set_trace()
        for Xi in full_seq_data_train:
            removed_points = torch.randperm(full_seq_data_train.size(1), generator=generator)[:int(full_seq_data_train.size(1) * missing_rate)].sort().values
            Xi[removed_points] = float('nan')
        for Xi in full_seq_data_test:
            removed_points = torch.randperm(full_seq_data_test.size(1), generator=generator)[:int(full_seq_data_test.size(1) * missing_rate)].sort().values
            Xi[removed_points] = float('nan')
    
    eval_length = full_seq_data_test.shape[0]

    train_seq_data = full_seq_data_train
    train_y_data = full_y_seq_data_train
    train_forecast_data = full_forecast_data_train
    train_forecast_seq = forecast_seq_train
    
    val_seq_data = full_seq_data_test[:int(eval_length*0.5)]
    val_y_data = full_y_seq_data[:int(eval_length*0.5)]
    val_forecast_data = full_forecast_data_test[:int(eval_length*0.5)]
    val_forecast_seq = forecast_seq_test[:int(eval_length*0.5)]
    
    test_seq_data = full_seq_data_test[int(eval_length*0.5):]
    test_y_data = full_y_seq_data[int(eval_length*0.5):]
    test_forecast_data = full_forecast_data_test[int(eval_length*0.5):]
    test_forecast_seq = forecast_seq_test[int(eval_length*0.5):]
    
    torch.save(train_seq_data,DATA_PATH_SAVE+'/train_seq_data.pt')
    torch.save(train_y_data,DATA_PATH_SAVE+'/train_y_data.pt')
    torch.save(train_forecast_data,DATA_PATH_SAVE+'/train_forecast_y.pt')
    torch.save(train_forecast_seq,DATA_PATH_SAVE+'/train_forecast_seq.pt')
    

    torch.save(val_seq_data,DATA_PATH_SAVE+'/val_seq_data.pt')
    torch.save(val_y_data,DATA_PATH_SAVE+'/val_y_data.pt')
    torch.save(val_forecast_data,DATA_PATH_SAVE+'/val_forecast_y.pt')
    torch.save(val_forecast_seq,DATA_PATH_SAVE+'/val_forecast_seq.pt')

    torch.save(test_seq_data,DATA_PATH_SAVE+'/test_seq_data.pt')
    torch.save(test_y_data,DATA_PATH_SAVE+'/test_y_data.pt')
    torch.save(test_forecast_data,DATA_PATH_SAVE+'/test_forecast_y.pt')
    torch.save(test_forecast_seq,DATA_PATH_SAVE+'/test_forecast_seq.pt')

    times = torch.Tensor(np.arange(look_window))
    torch.save(times,DATA_PATH_SAVE+'/times.pt')
    return input_size

In [30]:
here = os.getcwd()
def get_data(look_window,forecast_window,stride_window,missing_rate,learning_method='self-supervised'):
    
    
    base_base_loc = here / 'processed_data'
    if learning_method=='unsupervised':
        if missing_rate ==0:
            loc = base_base_loc / ('look_'+str(look_window)+'_stride_'+str(stride_window)+'_forecast_'+str(forecast_window))
        # loc = base_base_loc / ('EXPSMD_121_look_'+str(look_window)+'_stride_'+str(stride_window)+'_Missing_'+str(missing_rate))
        else:
            loc = base_base_loc / ('look_'+str(look_window)+'_stride_'+str(stride_window)+'_forecast_'+str(forecast_window)+'_Missing_'+str(missing_rate))
    
    else:
        if missing_rate ==0:
            loc = base_base_loc / ('look_'+str(look_window)+'_stride_'+str(stride_window)+'_forecast_'+str(forecast_window))
        # loc = base_base_loc / ('EXPSMD_121_look_'+str(look_window)+'_stride_'+str(stride_window)+'_Missing_'+str(missing_rate))
        else:
            loc = base_base_loc / ('look_'+str(look_window)+'_stride_'+str(stride_window)+'_forecast_'+str(forecast_window)+'_Missing_'+str(missing_rate))
    if os.path.exists(loc):
        PATH = os.path.dirname(os.path.abspath(__file__))
        data_path = PATH + '/data'
        # import pdb ; pdb.set_trace()
        X_times = normalized_train_data
        input_size = X_times.shape[1]
        # pass
    else:
        # download()
        
        if not os.path.exists(base_base_loc):
            os.mkdir(base_base_loc)
        if not os.path.exists(loc):
            os.mkdir(loc)
        input_size = _process_data(look_window,forecast_window,stride_window,missing_rate,loc,learning_method)
        
    return loc,input_size

In [31]:
# Load and run parse2.py file
%run parse2.py

In [32]:
from sklearn.metrics import precision_recall_fscore_support

In [33]:
CUBICS = ['natural_cubic','cubic']
torch.backends.cudnn.benchmark = True

In [34]:
%run common.py

In [35]:
%run iotDataset.py

In [38]:
%run pad.py

Namespace(seed=112, model='ncde', h_channels=64, hh_channels_f=128, hh_channels_g=64, hh_channels_c=64, lr=0.01, weight_decay=0.0001, epoch=5, solver_method='rk4', data_path='dataset/SWAT', dataset='SWAT', batch_size=256, win_size=30, forecast_window=10, step_size=30, interpolation='cubic', missing_rate=0)
18
======= >>> Model Info <<< =======
NeuralDE(
  (func_f): CDEFunc_f(
    (linear0): Linear(in_features=64, out_features=128, bias=True)
    (linear1): Linear(in_features=128, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=128, bias=True)
    (linear3): Linear(in_features=128, out_features=128, bias=True)
    (linear4): Linear(in_features=128, out_features=1152, bias=True)
  )
  (func_g): CDEFunc_g(
    (linear0): Linear(in_features=64, out_features=64, bias=True)
    (linear1): Linear(in_features=64, out_features=64, bias=True)
    (linear2): Linear(in_features=64, out_features=64, bias=True)
    (linear3): Linear(in_features=64, out_features=64, b